In [ ]:
# target = 'diff_RECEIPT_DATE_ ,  PO_FIRST_APPROVED_DATE'

In [1]:
try:
    import pandas as pd
    #from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import KFold
    import datetime as dt
    pd.set_option('display.max_columns',None)
    
    import xgboost as xgb
    

    from sklearn.metrics import r2_score,mean_squared_error
    import math
    import time

    import warnings
    warnings.filterwarnings('ignore')
    
    print('libraries are found')
except:
    print('Some libraries need to be installed')

libraries are found


In [2]:
def read_data(file_name):
    df = pd.read_hdf(file, key='data')
    return df

file = '/workspace/luca/Tabular/e2e/data/data_train_v7.h5'

df = read_data(file)

In [3]:
df.shape

(99563, 88)

In [4]:
df = df.dropna(thresh=df.shape[0]*0.5, how='all',axis=1) # -3
df = df.dropna(axis=0,subset=['RECEIPT_DATE','BOOKED_DATE'])
df.drop(['PROMISED_DATE','ORGANIZATION_CODE'],axis=1,inplace = True)
df = df[df['RECEIPT_DATE']>df['BOOKED_DATE']] # false
object_columns_df = df.select_dtypes(include=['object'])
df = df.dropna(axis=0,subset=list(object_columns_df.columns))
df = df.set_index('PO_STRING',drop=True)

In [5]:
df.shape

(73897, 82)

In [7]:
def analyze_data_types(df):
    date_columns_df = df.select_dtypes(include=['datetime64'])
    object_columns_df = df.select_dtypes(include=['object'])
    numerics= [ 'int64','float64']
    numeric_columns_df = df.select_dtypes(include = numerics)
    
    return date_columns_df, object_columns_df,numeric_columns_df

date_df, objects_df,numeric_df=analyze_data_types(df)

In [8]:
len(objects_df.columns)

27

In [9]:
objects_df.head()

,SUPPLIER,SUPPLIER_CODE,ITEM_CODE,ITEM_DESCRIPTION,PRODUCT_CODE,PRODUCT_DESCRIPTION,COE,COMMODITY,SUBCOMMODITY,SUPPLIER_SITE_COUNTRY,IMPORT_SOURCE,DESTINATION_CODE,TRANSPORTATION_LEAD_TIME,VENDOR_CODE,VENDOR_SITE_CODE,PO_EXCL,P_FLAG,MAKE_BUY,PEGFLAG_C09,PLANNER_CODE,FLOW_STATUS_CODE,ITEM_NUMBER,CONTR_SCO_LT2,IMP_SUPPLIER_VSCONTR,ODBL_IMPACT,PERFORMANCE_SUPPLIER,PERFORMANCE_PROC
PO_STRING,,,,,,,,,,,,,,,,,,,,,,,,,,,
440445102--2-1,OFFICINA MECCANICA MARIOTTI E C SRL,580261,SNO4582024,MARTINETTO PNEUMATICO * PER CRICCO,L7001,NO CRITICAL MACHINING,F2F,C-PARTS,MACHINED PARTS,IT,MSC,SalOrd,7,580261,IP02,NOT EXCL,Y,Buy,HARD,GS_PLAN140,CLOSED,SNO4582024,NO CONTR SCO,SUPPLIER ON TIME,NO ODB,SUPPLIER ON TIME - NOT PROC SCO,PROC LATE - PROC SCO
440095542-193-518-3,CMN COSTRUZIONI MECCANICHE SRL,576484,SWS1696000,ANELLO DI TENUTA* SW 1696000/LABYRINTH ...,L1405,CENTRIFUGAL COMPRESSOR LABYRINTH SEALS,F2F,C-PARTS,MACHINED PARTS,IT,MSC,SalOrd,7,576484,IP01,NOT EXCL,Y,Buy,HARD,GS_PLAN140,CLOSED,SWS1696000,NO CONTR SCO,SUPPLIER ON TIME,NO ODB,SUPPLIER ON TIME - NOT PROC SCO,PROC ON TIME - NOT PROC SCO
440448801--29-1,GE GLOBAL PARTS AND PRODUCTS GMBH HUNGARY,037111,0391-00-037,"O'RING,COVER",D4010,FR3-5 MIGRATED FLEET NON CAPITAL SPARE,F2F,GE TURBINE SPARE,FABRICATIONS,HU,MSC,SalOrd,14,037111,IP01,NOT EXCL,N,Buy,HARD,GS_PLAN140,CLOSED,0391-00-037,CONTR_SCO_LT2,SUPPLIER ON TIME,NO ODB,SUPPLIER ON TIME - PROC SCO,PROC LATE - PROC SCO
440252966-562-331-2,EUROGUARCO SPA,231266,RAZ14143,"GUARNIZIONE*$I 827,4",G1220,CIRCULAR FLAT GASKET (PLASTIC),F2F,C-PARTS,GASKETS,IT,MSC,SalOrd,7,231266,IP01,NOT EXCL,Y,Buy,HARD,GS_PLAN140,CLOSED,RAZ14143,NO CONTR SCO,SUPPLIER ON TIME,NO ODB,SUPPLIER ON TIME - NOT PROC SCO,PROC ON TIME - NOT PROC SCO
440438253--1-1,HONEYWELL SRL,394289,RJO21403,LIMIT SWITCH,F3903,LIMIT SWITCH,SI&EPC,ELECTRICAL & INSTRUMENTATION,SENSORS/INSTRUMENTS,IT,MSC,SalOrd,7,394289,IP02,NOT EXCL,N,Buy,SOFT,GS_PLAN232,CLOSED,RJO21403,NO CONTR SCO,LATE SUPPLIER,NO ODB,SUPPLIER LATE - NOT PROC SCO,PROC LATE - PROC SCO


In [10]:
# remove promised date.organizational code
for col in objects_df.columns:
    print('----------------')
    print(col,objects_df[col].nunique())

----------------
SUPPLIER 540
----------------
SUPPLIER_CODE 540
----------------
ITEM_CODE 41033
----------------
ITEM_DESCRIPTION 35169
----------------
PRODUCT_CODE 666
----------------
PRODUCT_DESCRIPTION 649
----------------
COE 13
----------------
COMMODITY 32
----------------
SUBCOMMODITY 98
----------------
SUPPLIER_SITE_COUNTRY 25
----------------
IMPORT_SOURCE 2
----------------
DESTINATION_CODE 97
----------------
TRANSPORTATION_LEAD_TIME 19
----------------
VENDOR_CODE 540
----------------
VENDOR_SITE_CODE 14
----------------
PO_EXCL 2
----------------
P_FLAG 2
----------------
MAKE_BUY 2
----------------
PEGFLAG_C09 2
----------------
PLANNER_CODE 14
----------------
FLOW_STATUS_CODE 4
----------------
ITEM_NUMBER 41033
----------------
CONTR_SCO_LT2 2
----------------
IMP_SUPPLIER_VSCONTR 2
----------------
ODBL_IMPACT 2
----------------
PERFORMANCE_SUPPLIER 4
----------------
PERFORMANCE_PROC 4


In [11]:
cols_cats_small = []
cols_cats_medium = []
cols_cats_large = []

list(objects_df.columns).remove('TRANSPORTATION_LEAD_TIME') 
for col in objects_df.columns:
    if (objects_df[col].nunique() <= 2): 
        cols_cats_small.append(col)
    elif (objects_df[col].nunique() <= 4):
            cols_cats_medium.append(col)
    else:
        
        cols_cats_large.append(col)
        
  
    

In [12]:
len(cols_cats_small+cols_cats_medium)

11

In [14]:
df1=df.copy()

In [15]:
df1.shape

(73897, 82)

In [17]:
df_s = df1[cols_cats_small]
df_m = df1[cols_cats_medium]
import category_encoders as ce
encoder_s = ce.BinaryEncoder(cols=cols_cats_small,return_df=True)
df_s = encoder_s.fit_transform(df_s)
encoder_m =ce.BaseNEncoder(cols=cols_cats_medium,return_df=True,base=3)
df_m = encoder_m.fit_transform(df_m)



In [21]:
list_remove = ['ITEM_DESCRIPTION', 'PRODUCT_DESCRIPTION']
cols_cats_large = list(set(cols_cats_large) - set(list_remove))

In [24]:
my_dict = {}
for col in cols_cats_large:
    values = df1[col].value_counts().to_dict()
    df1[col] = df1[col].map(values)


In [26]:
df1 = pd.concat([df1,df_s,df_m],axis=1)
df1.drop((cols_cats_small+cols_cats_medium),axis=1,inplace=True)  



In [33]:
# function to re-engineer the dates
def difference_dates(df,col1,col2):
            
    df['diff_'+col1+col2]=(df[col1]-df[col2]).dt.days
    df.drop([col1,col2],axis=1,inplace = True)
    
    return df



In [162]:
data =df1.copy()

In [163]:
# call function on the required features    
data = difference_dates(data,'RTS_','RTS_FINAL')
data = difference_dates(data,'SO_CWD','SO_CREATION_DATE')
data = difference_dates(data,'EOQ_CONTR','EOQ_REQUEST')
data = difference_dates(data,'CONTRACTUAL_DATE','FULFILLMENT_CONTRACTUAL_DATE')
data = difference_dates(data,'RECEIPT_DATE','PO_FIRST_APPROVED_DATE')  # target variable
#data = difference_dates(data,'RECEIPT_DATE','BOOKED_DATE') 
data = difference_dates(data,'PROMISED_DATE_LAST','PROMISED_DATE_FIRST')

In [164]:
# since most of the rows are same as NEED_BY_DATE','NEED_DATE_FIRST', 'NEED_DATE_LAST'
# 'NEED_BY_DATE' has same values as 'SUP_PO_NEED_BY_DATE'
# instead of these 3 features only one feature can be used and name it 'NEED_DATE_'
# 'DATE_WHEN_NEED_REV'
data['NEED_DATE_'] = data['NEED_BY_DATE']
data.drop(['NEED_DATE_FIRST', 'NEED_DATE_LAST','SUP_PO_NEED_BY_DATE','DATE_WHEN_NEED_REV',
         'NEED_BY_DATE'],axis=1,inplace=True)

# # both SCHEDULE_SHIP_DATE and 'LINE_REQUEST_DATE' are greater than NEED_DATE so take the diff
data['diff_SCHEDULE_SHIP_DATE_NEED_DATE_'] = (data['SCHEDULE_SHIP_DATE']-data['NEED_DATE_']).dt.days
data['diff_LINE_REQUEST_DATE_NEED_DATE_'] = (data['LINE_REQUEST_DATE']-data['NEED_DATE_']).dt.days

data.drop(['LINE_REQUEST_DATE','SCHEDULE_SHIP_DATE','BOOKED_DATE','POR_APPROVED_DATE'],axis=1,inplace=True)

data['NEED_DATE_numeric'] = data['NEED_DATE_'].map(dt.datetime.toordinal)
data.drop('NEED_DATE_',axis=1,inplace=True)
data['TRANSPORTATION_LEAD_TIME'] =data['TRANSPORTATION_LEAD_TIME'].astype(float)

In [165]:
# 'diff_CONTRACTUAL_DATEFULFILLMENT_CONTRACTUAL_DATE' retain this
def drop_features(df,cols_to_drop):
    df['diff_SO_LINE_FIRST_SO_LINE_LAST'] = df['SO_LINE_LAST'] - df['SO_LINE_FIRST']

    df.drop(cols_to_drop,axis=1,inplace=True)
    
    return df

cols_remove_1 = ['ORG_ID','ATTRIBUTE_NUMBER1','ATTRIBUTE_NUMBER2','ATTRIBUTE_NUMBER6','ATTRIBUTE_NUMBER7','ATTRIBUTE_NUMBER8',
                    'UNIT_PRICE_USD','SO_LINE_FIRST','SO_LINE_LAST']

cols_remove_2 = ['POSTPRO_LT_CALDAYS','PREPRO_LT_CALDAYS','CUMULATIVE_LT_CALDAYS', 'SO_LEADTIME_CALDAYS','RTSVSCONTR',
 'diff_RTS_RTS_FINAL', 'CONTRVSAPPROVED','FULL_LT_CALDAYS',
                 'LTPROC_SOLARDAYS', 'TOTALSTDLT_SOLARDAYS','LSSTNEEDVSRECEIPT']
cols_to_remove = cols_remove_1 + cols_remove_2

data = drop_features(data,cols_to_remove)

data=data.dropna()
data.shape

(65837, 63)

In [167]:
# 'FULL_LT_CALDAYS'
data.columns

Index(['SUPPLIER', 'SUPPLIER_CODE', 'ITEM_CODE', 'ITEM_DESCRIPTION',
       'PRODUCT_CODE', 'PRODUCT_DESCRIPTION', 'COE', 'LT2_STD', 'COMMODITY',
       'SUBCOMMODITY', 'SUPPLIER_SITE_COUNTRY', 'QUANTITY_ORDERED',
       'DESTINATION_CODE', 'AMOUNT_USD', 'TRANSPORTATION_LEAD_TIME',
       'VENDOR_ID', 'VENDOR_CODE', 'VENDOR_SITE_CODE', 'LID', 'PLANNER_CODE',
       'SUP_PO_LINE_LOCATION_IDDD', 'HEADER_ID', 'SO_NUMBER', 'LINE_NUMBER',
       'LINE_ID', 'SHIPMENT_NUMBER', 'LT2', 'ITEM_NUMBER', 'ATTRIBUTE_NUMBER3',
       'ATTRIBUTE_NUMBER5', 'ATTRIBUTE_NUMBER4', 'IMPORT_SOURCE_0',
       'IMPORT_SOURCE_1', 'PO_EXCL_0', 'PO_EXCL_1', 'P_FLAG_0', 'P_FLAG_1',
       'MAKE_BUY_0', 'MAKE_BUY_1', 'PEGFLAG_C09_0', 'PEGFLAG_C09_1',
       'CONTR_SCO_LT2_0', 'CONTR_SCO_LT2_1', 'IMP_SUPPLIER_VSCONTR_0',
       'IMP_SUPPLIER_VSCONTR_1', 'ODBL_IMPACT_0', 'ODBL_IMPACT_1',
       'FLOW_STATUS_CODE_0', 'FLOW_STATUS_CODE_1', 'PERFORMANCE_SUPPLIER_0',
       'PERFORMANCE_SUPPLIER_1', 'PERFORMANCE_PROC_0',

In [124]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack


In [95]:
# vectorizer1 = TfidfVectorizer(max_features=1000)
# vectorizer2 = TfidfVectorizer(max_features=10000)
# column_transformer = ColumnTransformer(
#     [('tfidf1', vectorizer1, 'ITEM_DESCRIPTION'), 
#     ('tfidf2', vectorizer2, 'PRODUCT_DESCRIPTION')],
#     remainder='passthrough')


In [168]:
target = 'diff_RECEIPT_DATEPO_FIRST_APPROVED_DATE'

X=data.drop(target,axis=1)
y=data[target]
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2,random_state=4)
print(train_X.shape,test_X.shape,train_y.shape,test_y.shape)

(52669, 61) (13168, 61) (52669,) (13168,)


In [169]:
train_X['ITEM_DESCRIPTION']=train_X['ITEM_DESCRIPTION'].str.lower()
train_X['PRODUCT_DESCRIPTION']=train_X['PRODUCT_DESCRIPTION'].str.lower()
train_X['ITEM_DESCRIPTION']=train_X['ITEM_DESCRIPTION'].replace('[^a-zA-Z0-9]', ' ',regex=True)
train_X['PRODUCT_DESCRIPTION']=train_X['PRODUCT_DESCRIPTION'].replace('[^a-zA-Z0-9]', ' ',regex=True)

In [170]:
vectorizer1 = TfidfVectorizer(max_features=100)
vectorizer2 = TfidfVectorizer(max_features=100)
column_transformer = ColumnTransformer(
    [('tfidf1', vectorizer1, 'ITEM_DESCRIPTION'), 
    ('tfidf2', vectorizer2, 'PRODUCT_DESCRIPTION')],
    remainder='passthrough')

In [129]:
# trainx = train_X.copy()
# testx  = test_X.copy()
# trainx1 = trainx.drop(['ITEM_DESCRIPTION','PRODUCT_DESCRIPTION'],axis=1)
# trainxt=trainx[['ITEM_DESCRIPTION','PRODUCT_DESCRIPTION']]
# trainxt['ITEM_DESCRIPTION']=trainxt['ITEM_DESCRIPTION'].str.lower()
# trainxt['PRODUCT_DESCRIPTION']=trainxt['PRODUCT_DESCRIPTION'].str.lower()
# trainxt['ITEM_DESCRIPTION']=trainxt['ITEM_DESCRIPTION'].replace('[^a-zA-Z0-9]', ' ',regex=True)
# trainxt['PRODUCT_DESCRIPTION']=trainxt['PRODUCT_DESCRIPTION'].replace('[^a-zA-Z0-9]', ' ',regex=True)
# vectorizer = TfidfVectorizer(max_features=100)
# X_tfidf = vectorizer.fit_transform(trainxt[['ITEM_DESCRIPTION','PRODUCT_DESCRIPTION']])

In [141]:
# if contractual date and fulfillment dates are not included these are the results

In [179]:
other_params={'learning_rate':0.1,'n_estimators':400,'max_depth':10,'min_child_weight':1,'subsample':0.6,'colsample_bytree':0.8}

regressor =xgb.XGBRegressor(objective='reg:squarederror',**other_params)

In [180]:
pipe = Pipeline([
                  ('tfidf', column_transformer),
                  ('classify', regressor)
                ])
pipe.fit(train_X,train_y)


Pipeline(steps=[('tfidf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidf1',
                                                  TfidfVectorizer(max_features=100),
                                                  'ITEM_DESCRIPTION'),
                                                 ('tfidf2',
                                                  TfidfVectorizer(max_features=100),
                                                  'PRODUCT_DESCRIPTION')])),
                ('classify',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.8, ear...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.1, max_bin=256,
                    

In [181]:
# with vectorizer tfidf 18 without contractuals,                   train error is 10  # need 13 and 0.95
# but with or without tfidf performance is same
predictions = pipe.predict(test_X)
print('Root_mean_squared_error_test set_',math.sqrt(mean_squared_error(test_y,predictions)))
print('R2 Score_test set_xgb',r2_score(test_y,predictions))

Root_mean_squared_error_test set_ 16.652283396209793
R2 Score_test set_xgb 0.9341645994113362


In [182]:

def eval_metrics(df,col_real, col_pred):
    print('accuracy 0days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 0) / results_df[col_real].shape[0])
    print('accuracy 3days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 3) / results_df[col_real].shape[0])
    print('accuracy 5days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 5) / results_df[col_real].shape[0])
    print('accuracy 7days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 7) / results_df[col_real].shape[0])
    print('accuracy 10days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 10) / results_df[col_real].shape[0])
    print('accuracy 20days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 20) / results_df[col_real].shape[0])
    print('accuracy 30days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 30) / results_df[col_real].shape[0])
    print('accuracy 90days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 90) / results_df[col_real].shape[0])

    print('************************************************************************************************')
    print('accuracy unpaired 30days', np.sum( ( (results_df[col_real] - results_df[col_pred])) <= 30) / results_df[col_real].shape[0])    
    
    return 



results_df = pd.DataFrame({'original_diff':test_y,'predicted_diff':predictions})
eval_metrics(results_df,'original_diff','predicted_diff')

accuracy 0days 0.0
accuracy 3days 0.30938639125151884
accuracy 5days 0.4474483596597813
accuracy 7days 0.5513365735115431
accuracy 10days 0.6732989064398542
accuracy 20days 0.8762150668286756
accuracy 30days 0.9455498177399757
accuracy 90days 0.9956713244228432
************************************************************************************************
accuracy unpaired 30days 0.9701549210206561


In [ ]:
# try randomized search later

# APPLY DNN

In [183]:
import tensorflow as tf
#tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.model_selection import RandomizedSearchCV


import pickle

In [ ]:
# basic dnn

In [184]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [191]:
train_XX = train_X.copy()
train_yy = train_y.copy()
test_XX  = test_X.copy()
test_yy  = test_y.copy()

In [194]:
columns = ['ITEM_DESCRIPTION', 'PRODUCT_DESCRIPTION']
dict_1 = {}
for col in columns:
    values_1 = train_XX[col].value_counts().to_dict()
    train_XX[col] = train_XX[col].map(values_1)
dict_2 = {}
for col in columns:
    values_2 = test_XX[col].value_counts().to_dict()
    test_XX[col] = test_XX[col].map(values_2)


In [198]:
# Try a simple XGB model
other_params={'learning_rate':0.1,'n_estimators':400,'max_depth':10,'min_child_weight':1,'subsample':0.6,'colsample_bytree':0.8}

regressor =xgb.XGBRegressor(objective='reg:squarederror',**other_params)

xgb_regressor =regressor.fit(train_XX, train_yy)
predictions = regressor.predict(test_XX)


print('Root_mean_squared_error_test set_',math.sqrt(mean_squared_error(test_yy,predictions)))
print('R2 Score_test set_xgb',r2_score(test_yy,predictions))

Root_mean_squared_error_test set_ 16.431703049094686
R2 Score_test set_xgb 0.9358971923732606


In [199]:
# func3
from sklearn.preprocessing import MinMaxScaler, StandardScaler
def scale_datasets(x_train, x_test):
    """
    Standard Scale test and train data
    Z - Score normalization
    """
    standard_scaler = StandardScaler()
    x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
    )
    x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
    )
    return x_train_scaled, x_test_scaled
# scale the dataset
X_train_scaled, X_test_scaled = scale_datasets(train_XX, test_XX)

In [ ]:
# see whether random search is helpful

In [207]:
from tensorflow import keras 
from keras import optimizers

In [200]:
X_train_scaled.shape

(52669, 61)

In [ ]:
X_train_scaled.columns #'LSSTNEEDVSRECEIPT','LTPROC_SOLARDAYS', 'TOTALSTDLT_SOLARDAYS','FULL_LT_CALDAYS'

## Apply Random Search on Neural Network

In [218]:
def build_model( nl1=1, nl2=1,  nl3=1,nn1=256, nn2=128, nn3 = 64, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0, input_shape=[X_train_scaled.shape[1]]):
    
    
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    # for the firt layer we need to specify the input dimensions
    first=True
    
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    model.add(Dense(1))
    model.compile(loss='mae', optimizer= 'adam', metrics=['mae'],)
    return model

In [219]:
# model class to use in the scikit random search CV 
model = KerasRegressor(build_fn=build_model, epochs=10, batch_size=64, verbose=1)

In [220]:
# learning algorithm parameters
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# activation
activation=['relu']

# numbers of layers
#nl1 = [0,1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[256,128,64]
nn2=[128,64,32]
nn3=[64,32]

# dropout and regularisation
dropout = [0.05]
l1 = [0.001,0.0001]
l2 = [0.001,0.0001]

# dictionary summary
param_grid = dict(
                    nl2=nl2, nl3=nl3, nn1=nn1, nn2=nn2, nn3=nn3,
                    act=activation, l1=l1, l2=l2, lr=lr, decay=decay, dropout=dropout, 
                    input_shape=[X_train_scaled.shape[1]]
                 )

In [221]:
random_nn_model = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20)

In [222]:
import time
start_time =time.time()
random_result = random_nn_model.fit(X_train_scaled, train_y)
print(f'Time taken is {time.time()-start_time} ')

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START act=relu, decay=0, dropout=0.05, input_shape=61, l1=0.001, l2=0.001, lr=0.0001, nl2=3, nl3=1, nn1=128, nn2=64, nn3=32
Epoch 1/10
549/549 [==============================] - 3s 3ms/step - loss: 34.2106 - mae: 31.8438
Epoch 2/10
549/549 [==============================] - 2s 3ms/step - loss: 22.9961 - mae: 20.9243
Epoch 3/10
549/549 [==============================] - 2s 3ms/step - loss: 21.6821 - mae: 19.7197
Epoch 4/10
549/549 [==============================] - 2s 3ms/step - loss: 20.9526 - mae: 19.0688
Epoch 5/10
549/549 [==============================] - 2s 3ms/step - loss: 20.4340 - mae: 18.6084
Epoch 6/10
549/549 [==============================] - 2s 3ms/step - loss: 20.2092 - mae: 18.4282
Epoch 7/10
549/549 [==============================] - 2s 3ms/step - loss: 19.6007 - mae: 17.8535
Epoch 8/10
549/549 [==============================] - 2s 3ms/step - loss: 19.0454 - mae: 17.3228
Epoch 9/10
549/549 [====

Epoch 1/10
549/549 [==============================] - 2s 3ms/step - loss: 34.5918 - mae: 34.4188
Epoch 2/10
549/549 [==============================] - 2s 3ms/step - loss: 21.3847 - mae: 21.2094
Epoch 3/10
549/549 [==============================] - 2s 3ms/step - loss: 20.0490 - mae: 19.8748
Epoch 4/10
549/549 [==============================] - 2s 3ms/step - loss: 19.4566 - mae: 19.2826
Epoch 5/10
549/549 [==============================] - 2s 3ms/step - loss: 19.0314 - mae: 18.8567
Epoch 6/10
549/549 [==============================] - 2s 3ms/step - loss: 18.5808 - mae: 18.4047
Epoch 7/10
549/549 [==============================] - 2s 3ms/step - loss: 18.2238 - mae: 18.0454
Epoch 8/10
549/549 [==============================] - 2s 3ms/step - loss: 17.8578 - mae: 17.6768
Epoch 9/10
549/549 [==============================] - 2s 3ms/step - loss: 17.4448 - mae: 17.2607
Epoch 10/10
275/275 [==============================] - 1s 2ms/step - loss: 16.4899 - mae: 16.3023
[CV 1/3; 3/10] END act=relu, 

Epoch 1/10
549/549 [==============================] - 3s 3ms/step - loss: 28.9562 - mae: 28.4045
Epoch 2/10
549/549 [==============================] - 2s 3ms/step - loss: 19.8324 - mae: 19.3419
Epoch 3/10
549/549 [==============================] - 2s 3ms/step - loss: 18.7641 - mae: 18.2930
Epoch 4/10
549/549 [==============================] - 2s 3ms/step - loss: 18.1671 - mae: 17.7007
Epoch 5/10
549/549 [==============================] - 2s 3ms/step - loss: 17.6169 - mae: 17.1513
Epoch 6/10
549/549 [==============================] - 2s 3ms/step - loss: 17.2631 - mae: 16.7955
Epoch 7/10
549/549 [==============================] - 2s 3ms/step - loss: 16.7830 - mae: 16.3131
Epoch 8/10
549/549 [==============================] - 2s 3ms/step - loss: 16.2642 - mae: 15.7927
Epoch 9/10
549/549 [==============================] - 2s 3ms/step - loss: 16.1258 - mae: 15.6533
Epoch 10/10
275/275 [==============================] - 1s 2ms/step - loss: 14.9483 - mae: 14.4730
[CV 1/3; 5/10] END act=relu, 

Epoch 1/10
549/549 [==============================] - 2s 3ms/step - loss: 32.2059 - mae: 31.6845
Epoch 2/10
549/549 [==============================] - 2s 3ms/step - loss: 20.9336 - mae: 20.4249
Epoch 3/10
549/549 [==============================] - 2s 3ms/step - loss: 19.8351 - mae: 19.3330
Epoch 4/10
549/549 [==============================] - 2s 3ms/step - loss: 19.1241 - mae: 18.6246
Epoch 5/10
549/549 [==============================] - 2s 3ms/step - loss: 18.5581 - mae: 18.0563
Epoch 6/10
549/549 [==============================] - 2s 3ms/step - loss: 18.0533 - mae: 17.5450
Epoch 7/10
549/549 [==============================] - 2s 3ms/step - loss: 17.5847 - mae: 17.0660
Epoch 8/10
549/549 [==============================] - 2s 3ms/step - loss: 17.2583 - mae: 16.7325
Epoch 9/10
549/549 [==============================] - 2s 3ms/step - loss: 16.8984 - mae: 16.3634
Epoch 10/10
275/275 [==============================] - 1s 2ms/step - loss: 15.8893 - mae: 15.3411
[CV 1/3; 7/10] END act=relu, 

Epoch 1/10
549/549 [==============================] - 3s 3ms/step - loss: 34.4485 - mae: 34.2501
Epoch 2/10
549/549 [==============================] - 2s 3ms/step - loss: 22.4959 - mae: 22.3062
Epoch 3/10
549/549 [==============================] - 2s 3ms/step - loss: 20.8175 - mae: 20.6328
Epoch 4/10
549/549 [==============================] - 2s 3ms/step - loss: 19.9749 - mae: 19.7922
Epoch 5/10
549/549 [==============================] - 2s 3ms/step - loss: 19.5455 - mae: 19.3636
Epoch 6/10
549/549 [==============================] - 2s 3ms/step - loss: 19.0511 - mae: 18.8699
Epoch 7/10
549/549 [==============================] - 2s 3ms/step - loss: 18.8181 - mae: 18.6376
Epoch 8/10
549/549 [==============================] - 2s 3ms/step - loss: 18.4955 - mae: 18.3142
Epoch 9/10
549/549 [==============================] - 2s 3ms/step - loss: 18.1633 - mae: 17.9804
Epoch 10/10
275/275 [==============================] - 1s 2ms/step - loss: 16.2799 - mae: 16.0953
[CV 1/3; 9/10] END act=relu, 

823/823 [==============================] - 3s 3ms/step - loss: 25.8197 - mae: 25.2872
Epoch 2/10
823/823 [==============================] - 3s 3ms/step - loss: 19.2987 - mae: 18.8192
Epoch 3/10
823/823 [==============================] - 3s 3ms/step - loss: 18.1747 - mae: 17.7049
Epoch 4/10
823/823 [==============================] - 3s 3ms/step - loss: 17.4184 - mae: 16.9468
Epoch 5/10
823/823 [==============================] - 3s 3ms/step - loss: 16.5959 - mae: 16.1196
Epoch 6/10
823/823 [==============================] - 3s 3ms/step - loss: 15.9594 - mae: 15.4788
Epoch 7/10
823/823 [==============================] - 3s 3ms/step - loss: 15.6979 - mae: 15.2137
Epoch 8/10
823/823 [==============================] - 3s 3ms/step - loss: 15.3426 - mae: 14.8558
Epoch 9/10
823/823 [==============================] - 3s 3ms/step - loss: 15.0125 - mae: 14.5227
Epoch 10/10
823/823 [==============================] - 3s 3ms/step - loss: 14.7304 - mae: 14.2381
Time taken is 627.1410989761353 


In [223]:
# for 10 epochs
best_model_10e = random_result.best_estimator_
print(random_result.best_params_)


{'nn3': 64, 'nn2': 128, 'nn1': 256, 'nl3': 0, 'nl2': 3, 'lr': 0.0001, 'l2': 0.0001, 'l1': 0.0001, 'input_shape': 61, 'dropout': 0.05, 'decay': 1e-06, 'act': 'relu'}


In [224]:
#  50 epochs works better than 10 epochs
predictions_rs_nn_10e = best_model_10e.predict(X_test_scaled)
print("RMSE Test_x",np.sqrt(mean_squared_error(test_y,predictions_rs_nn_10e))) 
print("R2 score Test_x",r2_score(test_y,predictions_rs_nn_10e))

206/206 [==============================] - 0s 1ms/step
RMSE Test_x 20.98602539188902
R2 score Test_x 0.8954383846719461


In [ ]:
# for 10 epochs
def eval_metrics(df,col_real, col_pred):
    print('accuracy 0days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 0) / results_df[col_real].shape[0])
    print('accuracy 3days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 3) / results_df[col_real].shape[0])
    print('accuracy 5days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 5) / results_df[col_real].shape[0])
    print('accuracy 7days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 7) / results_df[col_real].shape[0])
    print('accuracy 10days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 10) / results_df[col_real].shape[0])
    print('accuracy 20days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 20) / results_df[col_real].shape[0])
    print('accuracy 30days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 30) / results_df[col_real].shape[0])
    print('accuracy 90days', np.sum( np.abs((results_df[col_pred]- results_df[col_real])) <= 90) / results_df[col_real].shape[0])

    print('************************************************************************************************')
    print('accuracy unpaired 30days', np.sum( ( (results_df[col_real] - results_df[col_pred])) <= 30) / results_df[col_real].shape[0])    
    
    return 



results_df = pd.DataFrame({'original_diff':test_y,'predicted_diff':predictions_rs_nn_10e})
eval_metrics(results_df,'original_diff','predicted_diff')

### remove these features now and repeat the above expts.

In [ ]:

cols = ['LSSTNEEDVSRECEIPT','LTPROC_SOLARDAYS', 'TOTALSTDLT_SOLARDAYS','FULL_LT_CALDAYS']

In [ ]:
target = 'diff_RECEIPT_DATEPO_FIRST_APPROVED_DATE'

X=df1.drop(target,axis=1)
y=df1[target]
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2,random_state=4)
print(train_X.shape,test_X.shape,train_y.shape,test_y.shape)

train_X.drop(cols,axis=1,inplace=True)
test_X.drop(cols,axis=1,inplace=True)

In [ ]:
# Try a simple XGB model 
other_params={'learning_rate':0.1,'n_estimators':300,'max_depth':8,'min_child_weight':1,'subsample':0.6,'colsample_bytree':0.8}

regressor =xgb.XGBRegressor(objective='reg:squarederror',**other_params)

xgb_regressor =regressor.fit(train_X, train_y)
predictions = regressor.predict(test_X)

# 
# 
print('Root_mean_squared_error_test set_',math.sqrt(mean_squared_error(test_y,predictions)))
print('R2 Score_test set_xgb',r2_score(test_y,predictions))

In [ ]:
print(train_X.shape,test_X.shape)

In [ ]:
# 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
def scale_datasets(x_train, x_test):
    """
    Standard Scale test and train data
    Z - Score normalization
    """
    standard_scaler = StandardScaler()
    x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
    )
    x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
    )
    return x_train_scaled, x_test_scaled
# scale the dataset
X_train_scaled, X_test_scaled = scale_datasets(train_X, test_X)

In [ ]:
X_train_scaled.shape

In [ ]:
train_X.shape[1]

In [ ]:
def build_model( nl1=1, nl2=1,  nl3=1,nn1=256, nn2=128, nn3 = 64, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0, input_shape=[train_X.shape[1]]):
    
    
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    # for the firt layer we need to specify the input dimensions
    first=True
    
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))
            
    model.add(Dense(1))
    model.compile(loss='mae', optimizer= 'adam', metrics=['mae'],)
    return model

In [ ]:
# model class to use in the scikit random search CV 
model = KerasRegressor(build_fn=build_model, epochs=50, batch_size=32, verbose=1)

# learning algorithm parameters
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# activation
activation=['relu', 'tanh']

# numbers of layers
nl1 = [0,1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[256,128,64]
nn2=[128,64,32]
nn3=[64,32]

# dropout and regularisation
dropout = [0.05]
l1 = [0.001,0.0001]
l2 = [0.001,0.0001]

# dictionary summary
param_grid = dict(
                    nl1=nl1, nl2=nl2, nl3=nl3, nn1=nn1, nn2=nn2, nn3=nn3,
                    act=activation, l1=l1, l2=l2, lr=lr, decay=decay, dropout=dropout, 
                    input_shape=[X_train_scaled.shape[1]]
                 )

In [ ]:
X_train_scaled.shape[1]

In [ ]:
random_nn_model = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20)

import time
start_time =time.time()
random_result = random_nn_model.fit(X_train_scaled, train_y)
print(f'Time taken is {time.time()-start_time} ')

In [ ]:
# for 50 epochs
best_model_50e_23f = random_result.best_estimator_
print(random_result.best_params_)


#  50 epochs works better than 10 epochs
predictions_rs_nn_50_23f = best_model_50e_23f.predict(X_test_scaled)
print("RMSE Test_x",np.sqrt(mean_squared_error(test_y,predictions_rs_nn_50_23f))) 
print("R2 score Test_x",r2_score(test_y,predictions_rs_nn_50_23f))

In [ ]:
#28 features
best_model_50e_28f

In [ ]:
predictions_rs_nn_28f = best_model_50e_28f.predict(X_test_scaled)

print("RMSE Test_x",np.sqrt(mean_squared_error(test_y,predictions_rs_nn_28f))) 
print("R2 score Test_x",r2_score(test_y,predictions_rs_nn_28f))

In [ ]:
act=tanh, decay=1e-09, dropout=0.05, input_shape=28, l1=0.0001, l2=0.001, lr=0.0001, nl1=1, nl2=0, nl3=0, nn1=64, nn2=128, nn3=32

In [ ]:
def build_model(nn1, nn2, nn3, lr, decay, l1, l2,
                act, dropout, input_shape):
    
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
    model.add(Dropout(dropout))
    
        
    model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
    model.add(Dropout(dropout))
    
         
    
    model.add(Dense(1))
    model.compile(loss='mae', optimizer= 'adam', metrics=['mae'],)
    return model


custom_model = build_model(64, 128, 64, 0.0001, 1e-06, 0.0001, 0.0001,
                'relu',0.05, train_X.shape[1]) 

In [ ]:
#custom_model.summary()

#model = KerasRegressor(build_fn= custom_model, epochs=50, batch_size=32, verbose=1)

In [ ]:
import time
start_time =time.time()
result = custom_model.fit(X_train_scaled, train_y,validation_split=0.2,epochs=50, batch_size=32, verbose=1)
print(f'Time taken is {time.time()-start_time} ')

In [ ]:
test_predictions = custom_model.predict(X_test_scaled).flatten()